# CNNs in TensorFlow

In [ ]:
k = 64

img_size = [10,10,3]

filter_size = [5,5]

import tensorflow as tf

# image 
input_img = tf.placeholder(tf.float32, shape=[None, *img_size])

# weight + bias
weight = tf.Variable(tf.truncated_normal([*filter_size, img_size[2], k]))
bias = tf.Variable(tf.zeros(k))

# hyperparams
strides = [1,2,2,1]
padding = 'SAME'

# apply convolution 
conv_layer = tf.nn.conv2d(input_img, weight, strides=strides, padding=padding)
# add bias
conv_layer = tf.nn.bias_add(conv_layer, bias)
# apply activation
conv_layer = tf.nn.relu(conv_layer)

## TensorFlow Max Pooling 

![max](image/max-pooling.png)

The image above is an example of max pooling with a 2x2 filter and stride of 2. The four 2x2 colors represent each time the filter was applied to find the maximum value.

Benefit of max pooling is to reduce size of the input and allows CNN to focus on only the most important elements (hence retaining only the max value for each filtered area and removing the rest) 

TF provides `tf.nn.max_pool()` function to apply max pooling to conv layers.

Parameters: 

- `ksize` size of the filter `[1,2,2,1]` is common
- `strides` `[1,2,2,1]` is common
- `padding` 

First and last dimensions in the ksize/stride array are for `batch` and `channels`, and are usually set to `1`. 

In [ ]:
conv_layer = tf.nn.max_pool(
    conv_layer,
    ksize = [1,2,2,1],
    strides = [1,2,2,1],
    padding = 'SAME')

## Full Example CNN in TensorFlow 

### Dataset 

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets(".", one_hot=True, reshape=False)

import tensorflow as tf

# Parameters 
learning_rate = 0.001
epochs = 100
batch_size = 128

# Samples for validation and test sets 
test_valid_size = 256

# Network params
num_classes = 10
dropout = .75

### Weights and Biases

In [ ]:
weights = {
    'conv1': tf.Variable(tf.random_normal([5,5,1,32])),
    'conv2': tf.Variable(tf.random_normal([5,5,32,64])),
    'fc1': tf.Variable(tf.random_normal([7*7*64,1024])),
    'out': tf.Variable(tf.random_normal([1024, num_classes]))
}

biases = {
    'conv1': tf.Variable(tf.random_normal([32])),
    'conv2': tf.Variable(tf.random_normal([64])),
    'fc1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([num_classes]))
}

### Convolutions

![](image/convolution-schematic.gif)

Convolution w/ 3x3 filter and stride of 1 being applied to data.

The convolution for each 3x3 section is calculated against the weight, `[[1, 0, 1], [0, 1, 0], [1, 0, 1]]`, then a bias is added to create the convolved feature on the right. In this case, the bias is zero.

In [ ]:
def conv2d(x, W, b, stride = 1, padding='SAME'):
    hidden = tf.nn.conv2d(x, W, strides=[1,stride,stride,1], padding=padding)
    hidden = tf.nn.bias_add(hidden, b)
    return tf.nn.relu(hidden)

### Max Pooling

![](image/maxpool.jpeg)

Max pooling with 2x2 filter and a stride of 2. The colors in put represent each time the filter was applied to create the max on the right. 

For example, `[[1, 1], [5, 6]]` becomes 6 and `[[3, 2], [1, 2]]` becomes 3.

In [ ]:
def maxpool2d(conv, size=2):
    return tf.nn.max_pool(
        conv,
        ksize=[1,size,size,1],
        strides=[1,size,size,1],
        padding='SAME')

### Model

![](image/arch.png)

We're going to create 3 layers alternating b/w convs and max pooling followed by a fully connected and output layer. The transformation with the new dimensions are shown in the comments.

In [ ]:
def conv_net(input_data, weights, biases, dropout):
    # Layer 1 - 28x28x1 to 14x14x32
    conv1 = conv2d(input_data, weights['conv1'], biases['conv1'])
    conv1 = maxpool2d(conv1, size=2)
    
    # Layer 2 - 14x14x32 to 7x7x64
    conv2 = conv2d(conv1, weights['conv2'], biases['conv2'])
    conv2 = maxpool2d(conv2, size=2)
    
    # FC layer - 7x7x64 to 1024
    flat_dim = weights['fc1'].get_shape().as_list()[0] #7x7x64
    fc1 = tf.reshape(conv2, [-1, flat_dim])
    fc1 = tf.add(tf.matmul(fc1, weights['fc1']), biases['fc1'])
    fc1 = tf.nn.relu(fc1)
    fc1 = tf.nn.dropout(fc1, keep_prob)
    
    # output layer - 1024 to 10
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

### Session

In [ ]:
# tf Graph input
x = tf.placeholder(tf.float32, [None, 28, 28, 1])
y = tf.placeholder(tf.float32, [None, num_classes])
keep_prob = tf.placeholder(tf.float32)

# model 
logits = conv_net(x, weights, biases, dropout)

# loss, optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

# accuracy
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

import math

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(epochs):
        num_batches = math.ceil(mnist.train.num_examples/batch_size)
        for batch in range(num_batches):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            sess.run(optimizer, feed_dict={x: batch_x, y: batch_y, keep_prob: dropout})
            
        loss = sess.run(cost, feed_dict={
            x: batch_x,
            y: batch_y,
            keep_prob: 1.})
        valid_accuracy = sess.run(accuracy, feed_dict={
            x: mnist.validation.images,
            y: mnist.validation.labels,
            keep_prob: 1.})

        print('Epoch {} Loss: {:>10.4f} Validation Accuracy: {:.6f}'.format(epoch, loss, valid_accuracy))
            
    test_accuracy = sess.run(accuracy, feed_dict={
        x: mnist.test.images,
        y: mnist.test.labels,
        keep_prob: 1.})

    print('Test Accuracy: {:.6f}'.format(test_accuracy))